In [1]:
import os
import json
from openai import OpenAI
import requests

In [2]:
client = OpenAI(
    api_key = 'sk-proj-IiVjJx2eXk4wp6ROc3HxT3BlbkFJwXA9l2qEWVuS3DTRTrpB'
  )

In [3]:
finchat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "what are the stock price of Google right now",
        }
    ],
    model="gpt-4o",
)
print(finchat_completion.choices[0].message.content)

I can't provide real-time data, including the current stock price of Google (now Alphabet Inc., traded under the tickers GOOGL and GOOG). To find up-to-date stock prices, you can check financial news websites, stock market apps, or financial search engines like Google Finance, Yahoo Finance, or Bloomberg.


In [4]:
finfunctions = [
    {
        "name": "get_stock_price",
        "description": "Get the stock price of a given company for the given period.",
        "parameters": {
            "type": "object",
            "properties": {
                "company_name": {
                    "type": "string",
                    "description": "company name mentioned in text. e.g., Microsoft, Google"
                },
                "periods": {
                    "type": "string",
                    "description": "How old should the stock price data be? e.g., for query:- give me stock price of given company, output should be 1d. for query:- give me current stock price of given company, output should be 1d. for query:- give me yesturdays stock price of given company , output should be 1d. for query:- give me stock of given company of 2 year in past, output should be 2y. for query:- give me stock of given company of 1 year in past, output should be 1y. for query:- give me stock of given company of 5 year in past, output should be 5y. for query:- give me stock of given company of 10 year in past, output should be 10y. for query:- give me stock of given company of 1 month in past, output should be 1mo. for query:- give me stock of given company of 3 month in past, output should be 3mo. for query:- give me stock of given company of 6 month in past, output should be 6mo. for query:- give me stock of given company of 5 day in past, output should be 5d. for query:- give me stock of given company of 1 day in past, output should be 1d."
                }
            },
            "required": ["ticker", "periods"]
        }
    }
]

In [5]:
fin_prompt = 'what is the stock price of Google today?'

In [6]:
finchat_completion_after_functioncall = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": fin_prompt,
        }
    ],
    model = "gpt-4o",
    functions = finfunctions,
    function_call = "auto",
)
print(finchat_completion_after_functioncall)

ChatCompletion(id='chatcmpl-9UDmbWrEU1zdKazwDgG7dh3LSHhNY', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"company_name":"Google","periods":"1d"}', name='get_stock_price'), tool_calls=None))], created=1716989849, model='gpt-4o-2024-05-13', object='chat.completion', system_fingerprint='fp_43dfabdef1', usage=CompletionUsage(completion_tokens=22, prompt_tokens=358, total_tokens=380))


In [7]:
import yfinance as yf

def get_stock_price(company_name, periods):
    yfinance = "https://query2.finance.yahoo.com/v1/finance/search"
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    params = {"q": company_name, "quotes_count": 1, "country": "USA"}

    res = requests.get(url=yfinance, params=params, headers={'User-Agent': user_agent})
    data = res.json()

    company_code = data['quotes'][0]['symbol']
    
    stock = yf.Ticker(company_code)
    todays_data = stock.history(period=periods)
    currency = stock.info['currency']

    stock_info = {
        "stock_price": (todays_data['Close'][0] + todays_data['Open'][0])/2,
        "period": periods,
        "trade_currency": currency,
    }
    
    return json.dumps(stock_info)

In [8]:
finchat_completion_after_functioncall

ChatCompletion(id='chatcmpl-9UDmbWrEU1zdKazwDgG7dh3LSHhNY', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"company_name":"Google","periods":"1d"}', name='get_stock_price'), tool_calls=None))], created=1716989849, model='gpt-4o-2024-05-13', object='chat.completion', system_fingerprint='fp_43dfabdef1', usage=CompletionUsage(completion_tokens=22, prompt_tokens=358, total_tokens=380))

In [9]:
finoutput = finchat_completion_after_functioncall.choices[0].message
print(finoutput)

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"company_name":"Google","periods":"1d"}', name='get_stock_price'), tool_calls=None)


In [10]:
tickersymbol = json.loads(finoutput.function_call.arguments).get("company_name")
print(tickersymbol)

Google


In [11]:
finparams = json.loads(finoutput.function_call.arguments)
finparams

{'company_name': 'Google', 'periods': '1d'}

In [12]:
chosen_finfunction = eval(finoutput.function_call.name)
stockprice_param = chosen_finfunction(**finparams)
print(chosen_finfunction,stockprice_param)

<function get_stock_price at 0x00000291F25865C0> {"stock_price": 176.86019897460938, "period": "1d", "trade_currency": "USD"}


C:\Users\SURINDER\AppData\Local\Temp\ipykernel_15028\1540998670.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "stock_price": (todays_data['Close'][0] + todays_data['Open'][0])/2,


In [13]:
finchat_completion_stockprice = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": fin_prompt,
        },
        {
            "role": "function",
            "name": finoutput.function_call.name,
            "content": stockprice_param
        }
    ],
    model = "gpt-3.5-turbo",
    functions = finfunctions,
    function_call = "auto",
)
print(finchat_completion_stockprice.choices[0].message.content)

The stock price of Google today is $176.86 USD.


In [79]:
import yfinance as yf

tickerSymbol = 'GOOG'
tickerData = yf.Ticker(tickerSymbol)
tickerDf = tickerData.history(period='1d')

print(tickerDf)

                                 Open        High         Low       Close  \
Date                                                                        
2024-05-21 00:00:00-04:00  178.399994  179.820007  177.309998  179.539993   

                             Volume  Dividends  Stock Splits  
Date                                                          
2024-05-21 00:00:00-04:00  14697100        0.0           0.0  


In [80]:
import requests
def get_ticker(company_name):
    yfinance = "https://query2.finance.yahoo.com/v1/finance/search"
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    params = {"q": company_name, "quotes_count": 1, "country": "United States"}

    res = requests.get(url=yfinance, params=params, headers={'User-Agent': user_agent})
    data = res.json()

    company_code = data['quotes'][0]['symbol']
    return company_code

In [83]:
get_ticker("Maruti Suzuki")

'MARUTI.NS'